In [1]:
import argparse
import time
import torch
print(torch.__version__)
from Models import get_model
from Process import *
import torch.nn.functional as F
from Optim import CosineWithRestarts
from Batch import create_masks
import dill as pickle
import time
import math
import numpy as np
import multiprocessing as mp
import random
import string
import sys
import os
import whoosh, glob, time, pickle
import whoosh.fields as wf
from whoosh.qparser import QueryParser
from whoosh import index
import threading
from whoosh import filedb
from whoosh.filedb.filestore import FileStorage
import numpy as np
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing as mp
import time
from InMemorySearch import *
from client import *
from pathlib import Path

1.0.0


In [2]:
#At first checking our super clients
query_list = ["european crime records", "crime records", "european crime", "european records", "crime crimes violent sheriff enforcement re criminals stresak bill strikes", "LA times corpus", "crime records"]    
super_client = SuperClient()
print(super_client.hosts)
final_result = super_client.query_expansion_distributed(query_list)
print(len(final_result))

['10.141.0.104', '10.141.0.146', '10.141.0.134', '10.141.0.120', '10.141.0.121']


Process Process-2:
Process Process-3:
Traceback (most recent call last):
Process Process-4:
Traceback (most recent call last):
  File "/home/smsarwar/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/smsarwar/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/smsarwar/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/smsarwar/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/nfs/work1/allan/smsarwar/material/pytorch_transformer/client.py", line 38, in execute_on_server
    client.connect(host, port).send(query_dict)
  File "/mnt/nfs/work1/allan/smsarwar/material/pytorch_transformer/client.py", line 38, in execute_on_server
    client.connect(host, port).send(query_dict)
  File "/mnt/nfs/work1/allan/smsarwar/material/pytorc

2


In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [4]:
parser = argparse.ArgumentParser()
#If we are working on small dataset
small = 0
#If we want to activate relevance based training
relevance_training = 0
    
if small==1:
    parser.add_argument('-src_data', type=str, default='data/italian_small.txt')
    parser.add_argument('-trg_data', type=str, default='data/english_small.txt')
    parser.add_argument('-trg_data_retrieval', type=str, default='data/english_retrieval.txt')

else:
    parser.add_argument('-src_data', type=str, default='data/italian.txt')
    parser.add_argument('-trg_data', type=str, default='data/english.txt')  
    parser.add_argument('-trg_data_retrieval', type=str, default='data/LATIMESTEXT2.txt')

parser.add_argument('-src_lang', type=str, default='it')
parser.add_argument('-trg_lang', type=str, default='en')
parser.add_argument('-no_cuda', action='store_true')
parser.add_argument('-SGDR', action='store_true')
parser.add_argument('-epochs', type=int, default=1)
parser.add_argument('-d_model', type=int, default=200)
parser.add_argument('-n_layers', type=int, default=6)
parser.add_argument('-heads', type=int, default=8)
parser.add_argument('-dropout', type=int, default=0.1)
parser.add_argument('-batchsize', type=int, default=1000)
parser.add_argument('-printevery', type=int, default=50)
parser.add_argument('-load_vocab', type=str, default='clir_it_en')

if relevance_training == 1:
    my_file = Path("weights/model_weights")
    if my_file.is_file():
        parser.add_argument('-load_weights', type=str, default='weights')
    else:
        parser.add_argument('-load_weights', type=str, default=None)
    parser.add_argument('-lr', type=int, default=0.01)
else: 
    my_file = Path("weights/model_weights")
    if my_file.is_file():
        parser.add_argument('-load_weights', type=str, default='weights')
    else:
        parser.add_argument('-load_weights', type=str, default=None)         
    parser.add_argument('-lr', type=int, default=0.0001)
    
parser.add_argument('-create_valset', action='store_true')
parser.add_argument('-max_strlen', type=int, default=80)
parser.add_argument('-floyd', action='store_true')
parser.add_argument('-checkpoint', type=int, default=5)

opt = parser.parse_args(args=[])


In [5]:
def tokenizer(text):  # create a tokenizer function
    return text.split()
    
def create_fields(opt):    
    print("loading tokenizers...") 
    TRG = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>')
    SRC = data.Field(lower=True, tokenize=tokenizer)   
    SRC = pickle.load(open(f'{opt.load_vocab}/SRC.pkl', 'rb'))
    TRG = pickle.load(open(f'{opt.load_vocab}/TRG.pkl', 'rb'))
    return(SRC, TRG)

#this function will consider both europarl and CLEF
def create_dataset(opt, SRC, TRG):
    print("creating dataset and iterator... ")
    
    raw_data = {'src' : [line for line in opt.src_data], 'trg': [line for line in opt.trg_data]}
    df = pd.DataFrame(raw_data, columns=["src", "trg"])

    mask = (df['src'].str.count(' ') < opt.max_strlen) & (df['trg'].str.count(' ') < opt.max_strlen)
    df = df.loc[mask]

    df.to_csv("translate_transformer_temp.csv", index=False)

    data_fields = [('src', SRC), ('trg', TRG)]
    train = data.TabularDataset('./translate_transformer_temp.csv', format='csv', fields=data_fields)

    train_iter = MyIterator(train, batch_size=opt.batchsize, device=opt.device,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=True, shuffle=True)
      
    os.remove('translate_transformer_temp.csv')   
    
    if opt.load_vocab is None:
        print("creating dataset for retrieval corpus... ")
        raw_data = {'trg': [line for line in opt.trg_data_retrieval]}
        df = pd.DataFrame(raw_data, columns=["trg"])
        mask = (df['trg'].str.count(' ') > 1)
        df = df.loc[mask]
        df.to_csv("translate_transformer_retrieval_temp.csv", index=False)
        data_fields = [('trg', TRG)]
        train_retrieval = data.TabularDataset('./translate_transformer_retrieval_temp.csv', format='csv', fields=data_fields)
        os.remove('translate_transformer_retrieval_temp.csv')    

        print("building vocabulary for both europarl and retrieval corpus")

        SRC.build_vocab(train)
        TRG.build_vocab(train, train_retrieval)

    opt.src_pad = SRC.vocab.stoi['<pad>']
    opt.trg_pad = TRG.vocab.stoi['<pad>']
    opt.train_len = get_len(train_iter)
    return train_iter



In [6]:
opt.device = 0 if opt.no_cuda is False else -1
if opt.device == 0:
    assert torch.cuda.is_available()
print(opt.device)
read_data(opt)

SRC, TRG = create_fields(opt)
opt.train = create_dataset(opt, SRC, TRG)
dst = ''
#TRG = create_retrieval_vocabulary(opt, TRG)
model = get_model(opt, len(SRC.vocab), len(TRG.vocab))

opt.optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.98), eps=1e-9)

if opt.SGDR == True:
    opt.sched = CosineWithRestarts(opt.optimizer, T_max=opt.train_len)

if opt.checkpoint > 0:
    print("model weights will be saved every %d minutes and at end of epoch to directory weights/"%(opt.checkpoint))
    
if opt.load_vocab is None:
    pickle.dump(SRC, open(f'{dst}/SRC.pkl', 'wb'))
    pickle.dump(TRG, open(f'{dst}/TRG.pkl', 'wb'))

0
loading tokenizers...
creating dataset and iterator... 


The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


loading pretrained weights...
model weights will be saved every 5 minutes and at end of epoch to directory weights/


In [7]:
super_client = SuperClient()

In [8]:
def train_model(model, opt, SRC, TRG):
    torch.cuda.empty_cache()
    inmem = WhooshInMemorySearch()
    print("training model...")
    model.train()
    start = time.time()
    if opt.checkpoint > 0:
        cptime = time.time()
                    
    for epoch in range(opt.epochs):
        total_loss = 0
        if opt.floyd is False:
            print("   %dm: epoch %d [%s]  %d%%  loss = %s" %\
            ((time.time() - start)//60, epoch + 1, "".join(' '*20), 0, '...'), end='\r')
        
        if opt.checkpoint > 0:
            torch.save(model.state_dict(), 'weights/model_weights')
                    
        for i, batch in enumerate(opt.train):
            torch.cuda.empty_cache()
            start = time.clock()
            src = batch.src.transpose(0,1) # src_size = (187, 4)
            trg = batch.trg.transpose(0,1) # trg_size = (187, 8)             
            if relevance_training == 1:
                trg_strings = [' '.join([TRG.vocab.itos[ind] for ind in ex]) for ex in trg]
                trg_strings_rm = super_client.query_expansion_distributed(trg_strings)
                
                trg_strings_rm_id = torch.LongTensor([[TRG.vocab.stoi[token] for token in sentence] for sentence in trg_strings_rm]).cuda()            
                
                embed_trg = model.decoder.embed(trg_strings_rm_id)
                #old loss
                #embed_trg = embed_trg.view(embed_trg.size(0), -1)
                #new loss 
                embed_trg = embed_trg.sum(1)
            
            trg_input = trg[:, :-1]
            src_mask, trg_mask = create_masks(src, trg_input, opt)            
            src_mask = src_mask.cuda()            
            trg_mask = trg_mask.cuda()
            src = src.cuda() 
            trg_input = trg_input.cuda()                                  
            preds = model(src, trg_input, src_mask, trg_mask)
            
            #the goal is to find the embedding of the predictions 
            if relevance_training == 1: 
                out = F.softmax(preds, dim=-1)
                probs, ix = out[:, :].data.topk(1)
                preds_token_ids = ix.view(ix.size(0), -1)
                #new loss function 
                embed_pred = model.decoder.embed(preds_token_ids)
                embed_pred = embed_pred.sum(1)
                #print("embed preds size " + str(embed_pred_sum.size()))            

                #old loss function
#                 pred_strings = [' '.join([TRG.vocab.itos[ind] for ind in ex]) for ex in preds_token_ids]
#                 pred_strings_rm = super_client.query_expansion_distributed(pred_strings)                                
#                 pred_strings_rm_id = torch.LongTensor([[TRG.vocab.stoi[token] for token in sentence] for sentence in pred_strings_rm]).cuda()            
#                 embed_pred = model.decoder.embed(pred_strings_rm_id)
#                 embed_pred = embed_pred.view(embed_pred.size(0), -1)            
                     
            
            ys = trg[:, 1:].contiguous().view(-1).cuda()
            
            opt.optimizer.zero_grad()
            
            if relevance_training==1:
                #loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=opt.trg_pad).add(
                #((embed_trg.cuda() - embed_pred.cuda()) **2).mean())
                loss1 = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=opt.trg_pad)              
                loss2 = ((embed_trg.cuda() - embed_pred.cuda()) **2).mean()
                print("batch loss nmt\t" + str(loss1.item()) + "\tbatch loss relevance\t" + str(loss2.item()))
                loss1.backward(retain_graph=True)
                loss2.backward()               
            else:
                loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=opt.trg_pad)              
                loss.backward()             
            
            opt.optimizer.step()
                
            if opt.SGDR == True: 
                opt.sched.step()
            #total_loss += loss1.item() + loss2.item()
            total_loss+= loss.item()
            
            if (i + 1) % opt.printevery == 0:
                 p = int(100 * (i + 1) / opt.train_len)
                 avg_loss = total_loss/opt.printevery
                 if opt.floyd is False:
                    print("   %dm: epoch %d [%s%s]  %d%%  loss = %.3f" %\
                    ((time.time() - start)//60, epoch + 1, "".join('#'*(p//5)), "".join(' '*(20-(p//5))), p, avg_loss), end='\r')
                 else:
                    print("   %dm: epoch %d [%s%s]  %d%%  loss = %.3f" %\
                    ((time.time() - start)//60, epoch + 1, "".join('#'*(p//5)), "".join(' '*(20-(p//5))), p, avg_loss))
                 total_loss = 0
            
            if opt.checkpoint > 0 and ((time.time()-cptime)//60) // opt.checkpoint >= 1:
                torch.save(model.state_dict(), 'weights/model_weights')
                cptime = time.time()
                        
        print("%dm: epoch %d [%s%s]  %d%%  loss = %.3f\nepoch %d complete, loss = %.03f" %\
        ((time.time() - start)//60, epoch + 1, "".join('#'*(100//5)), "".join(' '*(20-(100//5))), 100, avg_loss, epoch + 1, avg_loss))


In [9]:
#torch.cuda.empty_cache()
model = model.cuda()
train_model(model, opt, SRC, TRG)

training model...
25794681m: epoch 1 [####################]  100%  loss = 3.41313
epoch 1 complete, loss = 3.413


In [10]:
torch.save(model.state_dict(), f'{load_vocab}/model_weights')

# x = torch.randn(4,6,1)
# print (x)
# x = x.view(4, -1)
# print (x)
# # print(x)
# # probs, ix = x[:, :].data.topk(1)
# # print(probs)
# # print(ix)

# x = torch.randn(3,2)
# print(x)
# y = torch.randn(3,2)
# print(y)
# print (((x - y)**2).mean())
# #F.cosine_embedding_loss(x, y, reduction='mean')

NameError: name 'load_vocab' is not defined

In [ ]:
print(SRC.vocab.itos[4880])